ICPPB 2024 Workshop:
===
###### (Meta)Genomics for pathogen identification

Metagenome classification using custom taxonomy databases created with LINtax
===
###### Parul Sharma
###### July 2024

LINtax creates LIN (Life identification number) based taxonomy files for any given list of input genomes. Instead of traditional ranks, LIN-taxonomy is based on 20 thresholds/ranks of average nucleotide identity (ANI). These thresholds of similarity ranges from 70%-99.999%.

Traditional taxonomy considers 'species' as the lowest rank which corresponds to approximately 95% ANI. However, with LIN-taxonomy you get much more resolution and 14 more ranks at the sub-species level which allows for accurate outbreak level detection of pathogens.

In this tutorial we will cover:
- creating a custom database with lintax
- classifying metagenomes with Kraken2 and lintax

 `Ralstonia CASE STUDY` : Using LINtax to identify Select Agent Pathogen from "infected" plant metagenome samples 

 


We will analyze the following samples:
- `sample0` : control 
- `sampleII` : sample with select agent pathogen
- `sampleIV` : sample with non-select-agent but very closely related pathogen

-----------
#### Questions we are trying to answer:
-   Can we detect the pathogen in a metagenome?
-   Can we identify the correct strain?
----------

### Creating custom taxonomy with LINtax

Metagenome classifiers use taxonomy to improve classifications by reducing errors due to genome similarity. All taxonomic classifiers (metagenome classifiers that use taxonomy) make use of the NCBI Taxonomy database which retains the information in 2 files:

- `names.dmp` : stores all tax ranks and their names
- `nodes.dmp` : stores all the heirarchial relations between different ranks

Using LINtax, users can create this taxonomy file structure for their list of input. 

Example input-file looks omething like this:


| LIN                               | Species              | Strain    | FileName                                         | Accession       |
|-----------------------------------|----------------------|-----------|--------------------------------------------------|-----------------|
| 14,1,0,0,0,0,0,0,0,0,6,0,1,0,1,0,0,0,0,0 | Ralstoniasolanacearum | OE1_1     | GCF_001879565.1_ASM187956v1_genomic.fna | GCF_001879565.1 |
| 14,1,0,0,0,0,0,0,0,0,6,0,1,0,0,0,0,0,0,0 | Ralstoniasolanacearum | PSS1308   | GCF_001870805.1_ASM187080v1_genomic.fna | GCF_001870805.1 |
| 14,1,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0 | Ralstoniasolanacearum | FJAT_1458 | GCF_001887535.1_ASM188753v1_genomic.fna | GCF_001887535.1 |
| 14,1,0,0,0,0,0,0,0,0,2,0,0,4,4,0,0,0,0,0 | Ralstoniasolanacearum | Pe_13     | GCF_012062595.1_ASM1206259v1_genomic.fna | GCF_012062595.1 |



In this tutorial, we will use a pre-computed database composed of 32 Ralstonnia strains that captures the diversity of this species complex.

Let's download the database:

%%bash 
wget -O kraken2-db.tar.gz https://osf.io/u4tq8/download


Extract the contents of the file

In [28]:
%%bash
tar -xvf kraken2-db.tar.gz


Now that we have the database, lets have a look at the contents of the file:

In [2]:
!ls kraken2-db

!ls kraken2-db/taxonomy/


hash.k2d  opts.k2d  seqid2taxid.map  taxo.k2d  taxonomy
data.txt  names.dmp  nodes.dmp	prelim_map.txt


THe Database has sequence-to-taxnonomy mappings for our input genomes and the taxonomy folder has the nodes.dmp and the names.dmp file! 

With the database all ready to go, let's do the rest of the set-up.

We need 3 more things:
- `LINgroups file` - download this file that contains all the taxonomic groups we want to identify and their corresponsding LIN prefixes
- `lintax tool` - install lintax
- `env` - set-up conda env for our analysis

In [29]:
%%bash
# install github repo
git clone https://github.com/parul-sharma/LINtax.git

# download lingroups file
wget -o LINgroups.txt https://osf.io/download/nqms2/

# install dependencies using the conda recipe: 
conda env create -f lintax.yaml
conda activate lintax


In [5]:
##OPTIONAL
## you can also export the LINtax/bin folder to the $PATH variable to access the scripts from anywhere
%%bash

export PATH="LINtax/bin:$PATH"


SyntaxError: invalid syntax (1549180392.py, line 4)

##### Lets start with sample0

since we have the pre-computed database, we can directly use the 'classify' function to classify the sample (We can also directly use kraken2 commands to classify and then use lintax report to get the final report):

In [6]:
%%bash
pwd
LINtax/bin/lintax classify -h


/workspaces/ICPPB2024

    ----------------------------------------------------------------------------------------
    LINtax: Create custom taxonomy for metagenome classification                        
    ----------------------------------------------------------------------------------------        
    Usage: lintax classify [options] <input_files>
    Options:
        --db                      LINtax Database to use for classifications (make sure to run build before this step) [REQUIRED]
        --lins                    Text file containing lingroups (custom taxa) and their LIN prefixes [REQUIRED]
        -o  or --output           LINreport containing the results [REQUIRED]
        -h  or --help             Show usage
        Other options:
        -c  or --confidence       Specify the confidence threshold for kraken classification [optional]
        --paired                  For paired-end reads
    Example: for single reads - 
        lintax classify --db /database/ --lins li

In [7]:
%%bash

LINtax/bin/lintax classify --db kraken2-db --lins LINgroups.txt -o sample0.LINreport -c 0.1 Sample-0.fasta


Loading database information... done.
9204 sequences (37.44 Mbp) processed in 2.539s (217.5 Kseq/m, 884.81 Mbp/m).
  0 sequences classified (0.00%)
  9204 sequences unclassified (100.00%)


In [8]:
%%bash

head sample0.LINreport


LINgroup_Name	LINgroup_prefix	Assigned_reads	Percentage_assigned_reads	Unique_Assigned_reads	Percentage_unique_assigned_reads	Total_reads_length
A_Total_reads;B_PhylI	14,1,0,0,0,0,0,0,0,0					
A_Total_reads;B_PhylI;C_seq14	14,1,0,0,0,0,0,0,0,0,3					
A_Total_reads;B_PhylI;C_seq15	14,1,0,0,0,0,0,0,0,0,2					
A_Total_reads;B_PhylI;C_seq34	14,1,0,0,0,0,0,0,0,0,6					
A_Total_reads;B_PhylII	14,1,0,0,0,3,0					
A_Total_reads;B_PhylII;C_IIA	14,1,0,0,0,3,0,1					
A_Total_reads;B_PhylII;C_IIC	14,1,0,0,0,3,0,2					
A_Total_reads;B_PhylII;C_IIB	14,1,0,0,0,3,0,0					
A_Total_reads;B_PhylII;C_IIB;D_seq1&seq2	14,1,0,0,0,3,0,0,0,0,1,0,0,0,0					


Try adding -c (or --confidence) option to 0.1. This equates to 10% confidence threshold and tells kraken2 to only accept those taxonomic ranks where this confidence threshold is met. [Read more about confidence thresholds] (https://github.com/DerrickWood/kraken2/wiki/Manual#confidence-scoring)

For LINtax, in general, recommended confidence threshold is 0.1 for long reads and 0.3-0.4 for short reads. 



### Now let's look at `sampleII`

We can run the same classify command to run sampleII with -c 0.1.

In [3]:
%%bash

LINtax/bin/lintax classify --db kraken2-db --lins LINgroups.txt -o sampleII.LINreport -c 0.1 Sample-II.fasta


Loading database information... done.
9204 sequences (37.73 Mbp) processed in 3.188s (173.2 Kseq/m, 710.08 Mbp/m).
  812 sequences classified (8.82%)
  8392 sequences unclassified (91.18%)



Let's look at the results of this sample

###### using `awk '$3 != 0'` to only select for those taxa where assigned reads were more than 0. 

In [4]:
%%bash

cat sampleII.LINreport | awk '$3 != 0'


LINgroup_Name	LINgroup_prefix	Assigned_reads	Percentage_assigned_reads	Unique_Assigned_reads	Percentage_unique_assigned_reads	Total_reads_length
A_Total_reads;B_PhylII	14,1,0,0,0,3,0	732	90.14778325123153	598	73.64532019704434	5557909
A_Total_reads;B_PhylII;C_IIB	14,1,0,0,0,3,0,0	134	16.502463054187192	79	9.729064039408868	708286
A_Total_reads;B_PhylII;C_IIB;D_seq1&seq2	14,1,0,0,0,3,0,0,0,0,1,0,0,0,0	16	1.9704433497536946	14	1.7241379310344827	115923
A_Total_reads;B_PhylII;C_IIB;D_seq1&seq2;E_seq1	14,1,0,0,0,3,0,0,0,0,1,0,0,0,0,0,0	2	0.24630541871921183	2	0.24630541871921183	13897
Total_reads		812				


We see that all assigned reads follow the same taxonomic lineage-- leading to Phylotype II sequevar1 !!! This is our infected sample with the Select Agent pathogen! 

Now let's look at sampleIV


In [1]:
%%bash

LINtax/bin/lintax classify --db kraken2-db --lins LINgroups.txt -o sampleIV.LINreport -c 0.1 Sample-IV.fasta


Loading database information... done.
9204 sequences (37.91 Mbp) processed in 3.206s (172.3 Kseq/m, 709.57 Mbp/m).
  789 sequences classified (8.57%)
  8415 sequences unclassified (91.43%)


Looking at the result the same as above:

In [2]:

%%bash 

cat sampleIV.LINreport | awk '$3!=0'


LINgroup_Name	LINgroup_prefix	Assigned_reads	Percentage_assigned_reads	Unique_Assigned_reads	Percentage_unique_assigned_reads	Total_reads_length
A_Total_reads;B_PhylIV	14,1,0,0,0,2,0,0,0	585	74.1444866920152	483	61.21673003802282	4654046
A_Total_reads;B_PhylIV;C_seq10	14,1,0,0,0,2,0,0,0,0,0,0	102	12.927756653992395	74	9.37896070975919	680687
Total_reads		789				


We find in this sample, all assigned reads follow the taxonomic lineage of Phylotype IV, sequevar 10 group --> This is not a Select agent pathogen. 

------------------------------------------------
------

In [5]:
%%bash
wget -O sample1.fastq.gz https://osf.io/gct93/download


--2024-07-02 17:48:10--  https://osf.io/gct93/download
Resolving osf.io (osf.io)... 35.190.84.173


Connecting to osf.io (osf.io)|35.190.84.173|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://files.osf.io/v1/resources/ux7td/providers/osfstorage/667c4cb97ba708018838d094?action=download&direct&version=1 [following]
--2024-07-02 17:48:10--  https://files.osf.io/v1/resources/ux7td/providers/osfstorage/667c4cb97ba708018838d094?action=download&direct&version=1
Resolving files.osf.io (files.osf.io)... 35.186.214.196
Connecting to files.osf.io (files.osf.io)|35.186.214.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347991736 (332M) [application/octet-stream]
Saving to: ‘sample1.fastq.gz’

     0K .......... .......... .......... .......... ..........  0% 4.18M 79s
    50K .......... .......... .......... .......... ..........  0% 9.34M 57s
   100K .......... .......... .......... .......... ..........  0% 15.0M 46s
   150K .......... .......... .......... .......... ..........  0% 10.2M 42s
   200K .......... .......... ..

In [3]:
%%bash
kraken2 --db kraken2-db \
        --report s1-report.txt \
        --output s1-output.txt \
        sample1.fastq.gz


Loading database information... done.
221600 sequences (333.26 Mbp) processed in 29.968s (443.7 Kseq/m, 667.25 Mbp/m).
  62496 sequences classified (28.20%)
  159104 sequences unclassified (71.80%)


In [5]:
%%bash
python report-lin.py --taxdata kraken2-db/taxonomy/data.txt \
    --k_report s1-report.txt \
        --k_output s1-output.txt \
            --lingroup LINgroups.txt \
                --output s1_LINreport.txt


In [6]:
%%bash
more s1_LINreport.txt


::::::::::::::
s1_LINreport.txt
::::::::::::::
LINgroup_Name,LINgroup_prefix,Assigned_reads,Percentage_assigned_reads,Unique_Assigned_reads,Percentage_unique_assigned_reads,Total_reads_length
A_Total_reads;B_PhylI,"14,1,0,0,0,0,0,0,0,0",340,0.5440348182283666,183,0.2928187403993856,551360
A_Total_reads;B_PhylI;C_seq14,"14,1,0,0,0,0,0,0,0,0,3",123,0.19681259600614437,10,0.016001024065540194,27491
A_Total_reads;B_PhylI;C_seq15,"14,1,0,0,0,0,0,0,0,0,2",19,0.030401945724526373,7,0.011200716845878136,20507
A_Total_reads;B_PhylI;C_seq34,"14,1,0,0,0,0,0,0,0,0,6",15,0.02400153609831029,0,0.0,0
A_Total_reads;B_PhylII,"14,1,0,0,0,3,0",59111,94.58365335381464,3750,6.000384024577573,3513522
A_Total_reads;B_PhylII;C_IIA,"14,1,0,0,0,3,0,1",310,0.496031746031746,0,0.0,0
A_Total_reads;B_PhylII;C_IIC,"14,1,0,0,0,3,0,2",54383,87.01836917562724,0,0.0,0
A_Total_reads;B_PhylII;C_IIB,"14,1,0,0,0,3,0,0",668,1.068868407578085,142,0.22721454173067077,279370
A_Total_reads;B_PhylII;C_IIB;D_seq1&seq2,"14,1,0,0,0,